In [103]:
class MatrixSolver:
    # compute gradinent of function f in point x with step h


    def grad(x, f, eps):
        m = len(x)
        grad = np.zeros(m)
        for i in range(m):
            x[i] += 1e-5
            f_plus_delta = f(x)
            x[i] -= 2 * 1e-5
            f_minus_delta = f(x)
            x[i] += 1e-5
            grad[i] = (f_plus_delta - f_minus_delta) / (2 * 1e-5)
        return grad

    # compute Jacobian of functions rs = (r_1...r_n), r_i = r_i(x_1...x_m)
    @staticmethod
    def findJacobian(rs, x, eps):
        n = np.size(rs)
        m = np.size(x)
        J = np.zeros((n, m))
        for i in range(n):
            J[i] = MatrixSolver.grad(x, rs[i], eps)
        return J

    # compute pseudo inverse of matrix X
    @staticmethod
    def pseudoInverse(x):
        return np.linalg.inv(x.T @ x) @ x.T

    # compute hessian of functions R^n -> R
    @staticmethod
    def findHessian(func, x, eps):
        def grad_f(x):
            return (MatrixSolver.grad(x, func, eps))

        return MatrixSolver.findJacobian([grad_f], x, eps)


In [125]:
def is_true(fgrad, eps) :
    answer=0
    for i in range(np.size(fgrad)):
        answer=numpy.sum(numpy.power(fgrad, 2))      
    if answer<eps : return True
    else : return False 

def next_H(H, y, s):
    I = np.eye(len(s), len(s)) # единичная матрица
    print(y.transpose @ s)
    g= 1.0/(y.transpose @ s)
    # вот тут падает впервые
    g_s_y_T=g @ (s @ y.transpose())
    g_y_s_t=g @ (y @ s.transpose())
    return ((I-g_s_y_T) @ (H @ (I-g_y_s_t))) +g @ (s @ s.transpose())

### BFGS

In [129]:
import numpy
from numpy import matmul
import scipy
import scipy.optimize
from math import sin
from math import cos
# для начального приближения можно выбрать единичную матрицу
# x - точка на текущей итерации
# H - гессиан
# func - функция, которую минимизируем

def bfgs(func, x, m_epoch, func_grad, eps):
    # или обратный гессиан в точке x0
    size = len(x)

    H = np.eye(size, size)
    i = 0
    points = [x]
    while (i<m_epoch):

        # находим направление поиска
        fgrad=MatrixSolver.grad(x, func, eps)
        # когда дебажила, уже вот тут у них градиент получается другим, почему блять?????
        if is_true(fgrad, 1e-5) : break
        p= -H @ fgrad

        #находим альфу, которая подходит условиям вольфа
        alpha = scipy.optimize.line_search(func, func_grad , x, p, c1=1e-4, c2=0.9)
        if alpha[0] is None:
            x_next = x + 1e-4 * p
        else:
            x_next = x + alpha[0] * p
        

        # переход на следующую итерацию
        y=MatrixSolver.grad(x_next, func, eps)-fgrad
        s=x_next-x
        H=next_H(H, y, s)
        points.append(x_next)
        x = x_next
        i+=1
    return(points, i)   

In [127]:
def f1(X):
    return 0.4*X[0] + 4.7*sin(X[0]) + 2

def f1_grad(X) :
    return 0.4+4.7*cos(X[0])

In [128]:
a = bfgs(f1, np.array([10], float), 1000, f1_grad, 0.00001)

[10.]
[10.00035436]


c:\Users\Julko\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)